<a href="https://colab.research.google.com/github/aruaru0/colab_notebook/blob/main/santa_2021_tsp_baseline_2500_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Santa Movie Challenge as Traveling Salesman Problem 
This notebook is a starter notebook for solving Kaggle's 2021 Santa movie challenge as a traveling salesman problem. To solve TSP, we will use Lin-Kernighan heuristic aka LKH. Their homepage is [here][1] and user guide is [here][2]

[1]: http://webhotel4.ruc.dk/~keld/research/LKH-3/
[2]: http://akira.ruc.dk/~keld/research/LKH/LKH-2.0/DOC/LKH-2.0_USER_GUIDE.pdf

In [2]:
import itertools
import pandas as pd
import numpy as np

!wget http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.7.tgz
!tar xvfz LKH-3.0.7.tgz
!cd LKH-3.0.7; make

--2021-11-21 14:51:56--  http://webhotel4.ruc.dk/~keld/research/LKH-3/LKH-3.0.7.tgz
Resolving webhotel4.ruc.dk (webhotel4.ruc.dk)... 130.225.220.230
Connecting to webhotel4.ruc.dk (webhotel4.ruc.dk)|130.225.220.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2316546 (2.2M) [application/x-gzip]
Saving to: ‘LKH-3.0.7.tgz’

LKH-3.0.7.tgz       100%[===================>]   2.21M   544KB/s    in 4.4s    

2021-11-21 14:52:01 (514 KB/s) - ‘LKH-3.0.7.tgz’ saved [2316546/2316546]

LKH-3.0.7/
LKH-3.0.7/pr2392.par
LKH-3.0.7/whizzkids96.atsp
LKH-3.0.7/Makefile
LKH-3.0.7/whizzkids96.par
LKH-3.0.7/pr2392.tsp
LKH-3.0.7/DOC/
LKH-3.0.7/README.txt
LKH-3.0.7/SRC/
LKH-3.0.7/SRC/Penalty_CVRPTW.c
LKH-3.0.7/SRC/RestoreTour.c
LKH-3.0.7/SRC/SolveKMeansSubproblems.c
LKH-3.0.7/SRC/IsCommonEdge.c
LKH-3.0.7/SRC/Penalty_TSPPD.c
LKH-3.0.7/SRC/ReadProblem.c
LKH-3.0.7/SRC/BestKOptMove.c
LKH-3.0.7/SRC/Distance_SPECIAL.c
LKH-3.0.7/SRC/Penalty_TSPDL.c
LKH-3.0.7/SRC/Penalty_PDPTW.c
LKH-3.0.7/

# Begin With Best n=7 Solution
We begin with the best published superpermutation for n=7 [here][1]

[1]: https://www.gregegan.net/SCIENCE/Superpermutations/Superpermutations.html

In [3]:
def merge_shift(s,t) :
    for k in range(len(s)) :
        ss = s[k:]
        tt = t[:len(ss)]
        if ss == tt :
            return s[:k+len(ss)] + t[len(ss):]
    return s+t

def is_perm(s,n):
    y = True
    for k in range(1,n+1):
        y = y&(str(k) in s)
        if not y: break
    return y

def supermutation(x, ns, ms):
    n = int(ns)
    p = []
    for k in range(len(x)-n+1) :
        t = x[k:k+n]
        if is_perm(t, n) :
            p.append(t+ms+t)
            #print(t)
    ret = ""
    print(p)
    for e in p:
        ret = merge_shift(ret, e)
    return ret    

In [4]:
a = supermutation('121','2','3')
a = supermutation(a,'3','4')
a = supermutation(a,'4','5')
a = supermutation(a,'5','6')
a = supermutation(a,'6','7')

['12312', '21321']
['1234123', '2314231', '3124312', '2134213', '1324132', '3214321']
['123451234', '234152341', '341253412', '412354123', '231452314', '314253142', '142351423', '423154231', '312453124', '124351243', '243152431', '431254312', '213452134', '134251342', '342153421', '421354213', '132451324', '324153241', '241352413', '413254132', '321453214', '214352143', '143251432', '432154321']
['12345612345', '23451623451', '34512634512', '45123645123', '51234651234', '23415623415', '34152634152', '41523641523', '15234615234', '52341652341', '34125634125', '41253641253', '12534612534', '25341625341', '53412653412', '41235641235', '12354612354', '23541623541', '35412635412', '54123654123', '23145623145', '31452631452', '14523614523', '45231645231', '52314652314', '31425631425', '14253614253', '42531642531', '25314625314', '53142653142', '14235614235', '42351642351', '23514623514', '35142635142', '51423651423', '42315642315', '23154623154', '31542631542', '15423615423', '54231654231', 

In [5]:
ch = dict({
    '1':'3',
    '2':'4',
    '3':'5',
    '4':'6',
    '5':'7',
    '6':'1',
    '7':'2',
})

sub = ''
for e in a :
    sub+=ch[e]

In [6]:
required_permutaions = ['12' + ''.join(x) for x in itertools.permutations(['3','4','5','6','7'], 5)]

In [7]:
len(sub)

5913

In [8]:
solution = sub
all = []
for k in range(len(solution)-7+1) :
    s = solution[k:k+7]
    if is_perm(s, 7) and s not in required_permutaions:
        all.append(s)

In [9]:
len(all)

4920

In [10]:
permutations = all
mandatory = required_permutaions

In [11]:
adjust1 = 0
adjust2 = 0
x =  len(all)//3
group1 = permutations[:x-adjust1] # + mandatory
group2 = permutations[x-adjust1:2*x+adjust2] # + mandatory
group3 = permutations[2*x+adjust2:] #+ mandatory


print(len(group1), len(group2), len(group3))
print(group1[:10],"...")
print(group2[:10],"...")
print(group3[:10],"...")

1640 1640 1640
['3456712', '4567123', '5671234', '6712345', '7123456', '2345671', '4567132', '5671324', '6713245', '7132456'] ...
['5346712', '3467125', '4671253', '6712534', '7125346', '2534671', '3467152', '4671523', '6715234', '7152346'] ...
['3546712', '5467123', '4671235', '6712354', '7123546', '2354671', '5467132', '4671325', '6713254', '7132546'] ...


In [12]:
def count(g) :
  cnt = 0
  for e in mandatory :
    if e in g :
      cnt+=1
  print(cnt)


# Helper Functions

In [13]:
def hamming_distance(str1, str2):
    return sum( (c1!=c2) for c1, c2 in zip(str1, str2))

def offset(s1, s2):
    assert(len(s1)==len(s2))
    ln = len(s1)
    j = ln
    for k in range(0,ln):
        if hamming_distance(s1[k:],s2[:7-k])==0:
            j=k
            break
    return j

def calc_dist(s, t) :
    for k in range(len(s)) :
      ss = s[k:]
      tt = t[:len(ss)]
      if ss == tt :
            res = s[:k+len(ss)] + t[len(ss):]
            return len(res) - len(s)
    return len(t)

In [14]:
def merge_shift(s,t) :
    for k in range(len(s)) :
        ss = s[k:]
        tt = t[:len(ss)]
        if ss == tt :
            return s[:k+len(ss)] + t[len(ss):]
    return s+t

In [25]:
def get_tsp_solution(group):
    # CREATE DISTANCE MATRIX
    SIZE = len(group)
    M = np.zeros((SIZE, SIZE), dtype='int8')
    for j in range(SIZE):
        #if j%25==0: print(j,', ',end='')
        for k in range(SIZE):
            M[j,k] = calc_dist(group[j],group[k])
            
    # WRITE PROBLEM FILE
    f = open(f'group.par','w')
    f.write("PROBLEM_FILE = ../distances.atsp\n")
    f.write("TOUR_FILE = ../output.txt\n")
    f.write(f"OPTIMUM = {SIZE}\n")
    f.write("MOVE_TYPE = 5\n")
    f.write("PATCHING_C = 3\n")
    f.write("PATCHING_A = 2\n")
    f.write("RUNS = 5\n")
    f.write("TIME_LIMIT = 3600\n") #seconds
    f.close()
    
    # WRITE PARAMETER FILE
    f = open(f'distances.atsp','w')
    f.write("NAME: distances\n")
    f.write("TYPE: ATSP\n")
    f.write("COMMENT: Asymmetric TSP\n")
    f.write(f"DIMENSION: {SIZE}\n")
    f.write("EDGE_WEIGHT_TYPE: EXPLICIT\n")
    f.write("EDGE_WEIGHT_FORMAT: FULL_MATRIX\n")
    f.write("EDGE_WEIGHT_SECTION\n")
    for j in range(SIZE):
        #if j%25==0: print(j,', ',end='')
        for k in range(SIZE):
            f.write(f"{M[j,k]:2d} ") 
        f.write("\n")
    f.close()
    
    # EXECUTE TSP SOLVER
    !cd LKH-3.0.7; ./LKH ../group.par
    
    # READ RESULTING ORDER
    with open('output.txt') as f:
        lines = f.readlines()
    for i,ln in enumerate(lines):
        if 'TOUR_SECTION' in ln: break
    perms = [int(x[:-1]) for x in lines[i+1:-2] ]
    
    # CREATE STRING
    # result = group[ perms[0]-1 ]
    # for k in range(1,len(perms)):
    #     s1 = group[ perms[k-1]-1 ]
    #     s2 = group[ perms[k]-1 ]
    #     d = offset(s1,s2)
    #     assert(d!=0)
    #     result += s2[-d:]
    result = ""
    for k in range(len(perms)) :
      result = merge_shift(result, group[ perms[k]-1 ])

    return result

# Solve with LKH-3 TSP Solver
We will solve the traveling salesman problem with Lin-Kernighan heuristic aka LKH. Their homepage is [here][1] and user guide is [here][2]. We can improve this notebook's LB score by adjusting the following parameters. Currently to make this notebook run quickly, we limit the solver to only 1 run of 120 seconds. We can also improve LB by using wildcards.

* adjust1 = 18
* adjust2 = 12
* RUNS = 1
* TIME_LIMIT = 120

[1]: http://webhotel4.ruc.dk/~keld/research/LKH-3/
[2]: http://akira.ruc.dk/~keld/research/LKH/LKH-2.0/DOC/LKH-2.0_USER_GUIDE.pdf

In [26]:
len(all)/6

820.0

In [ ]:
def print_header(x):
    print(); print()
    print('#'*25)
    print('### Computing String',x)
    print('#'*25); print()


def do(adjust1, adjust2) :
  # x =  len(all)//6
  # group1 = permutations[:x] + permutations[5*x:] + mandatory
  # group2 = permutations[x:2*x] + permutations[4*x:5*x]  + mandatory
  # group3 = permutations[2*x:3*x] + permutations[3*x:4*x] + mandatory
  x =  len(all)//3
  group1 = permutations[:x+adjust1] + mandatory
  group2 = permutations[x+adjust1:2*x+adjust2] + mandatory
  group3 = permutations[2*x+adjust2:] + mandatory
 
  print("group = ", len(group1), len(group2), len(group3))

  print_header(1)
  string1 = get_tsp_solution(group1) 
  print_header(2)
  string2 = get_tsp_solution(group2) 
  print_header(3)
  string3 = get_tsp_solution(group3) 

  print("RESULT: strings=", len(string1), len(string2), len(string3))
  return string1, string2, string3


maxstr = 100000
string1, string2, string3 = do(0, 0)
# string1, string2, string3 = do(240, 195)


group =  1760 1760 1760


#########################
### Computing String 1
#########################

PARAMETER_FILE = ../group.par
Reading PROBLEM_FILE: "../distances.atsp" ... done
ASCENT_CANDIDATES = 50
BACKBONE_TRIALS = 0
BACKTRACKING = NO
# BWTSP =
# CANDIDATE_FILE =
CANDIDATE_SET_TYPE = ALPHA
# DISTANCE =
# DEPOT =
# EDGE_FILE =
EXCESS = 0.000568182
EXTERNAL_SALESMEN = 0
EXTRA_CANDIDATES = 0 
EXTRA_CANDIDATE_SET_TYPE = QUADRANT
GAIN23 = YES
GAIN_CRITERION = YES
INITIAL_PERIOD = 1760
INITIAL_STEP_SIZE = 1
INITIAL_TOUR_ALGORITHM = WALK
# INITIAL_TOUR_FILE = 
INITIAL_TOUR_FRACTION = 1.000
# INPUT_TOUR_FILE = 
KICK_TYPE = 0
KICKS = 1
# MAX_BREADTH =
MAKESPAN = NO
MAX_CANDIDATES = 5 
MAX_SWAPS = 3520
MAX_TRIALS = 3520
# MERGE_TOUR_FILE =
MOVE_TYPE = 5 
# MTSP_MIN_SIZE =
# MTSP_MAX_SIZE =
# MTSP_OBJECTIVE =
# MTSP_SOLUTION_FILE = 
NONSEQUENTIAL_MOVE_TYPE = 9
OPTIMUM = 1760
# OUTPUT_TOUR_FILE = 
PATCHING_A = 2 
PATCHING_C = 3 
# PI_FILE = 
POPMUSIC_INITIAL_TOUR = NO
POPMUSIC_MAX_NEIGHBO

In [ ]:
# # READ RESULTING ORDER
# group = group3
# with open('output.txt') as f:
#     lines = f.readlines()
# for i,ln in enumerate(lines):
#     if 'TOUR_SECTION' in ln: break
# perms = [int(x[:-1]) for x in lines[i+1:-2] ]

# # CREATE STRING
# result = group[ perms[0]-1 ]
# for k in range(1,len(perms)):
#     s1 = group[ perms[k-1]-1 ]
#     s2 = group[ perms[k]-1 ]
#     d = offset(s1,s2)
#     assert(d!=0)
#     result += s2[-d:]

# print(group[perms[0]-1], group[perms[1]-1], group[perms[2]-1])
# len(result)

# Verify
Let's verify that all permutations are contained in at least one string. And let's verify that the mandatory sequences are contained in every string.

In [ ]:
all_permutations = [''.join(x) for x in itertools.permutations(['1','2','3','4','5','6','7'], 7)]

for p in all_permutations:
    if p not in string1 and p not in string2 and p not in string3:
        print(p)

In [ ]:
for p in mandatory:
    if p not in string1:
        print(p)
    if p not in string2:
        print(p)
    if p not in string3:
        print(p)

# Write Submission CSV

In [ ]:
# CONVERT NUMBERS TO EMOJIS
replace_dict = {
 '1': '🎅',
 '2': '🤶',
 '8': '🌟',
 '3': '🦌',
 '4': '🧝',
 '5': '🎄',
 '6': '🎁',
 '7': '🎀'}

for k,v in replace_dict.items():
    string1 = string1.replace(k, v)
    string2 = string2.replace(k, v)
    string3 = string3.replace(k, v)

In [ ]:
# WRITE SUBMISSION CSV
sub = pd.DataFrame()
sub['schedule'] = [string1, string2, string3]
sub.to_csv('submission.csv',index=False)
sub.head()

In [ ]:
print(len(string1))
print(len(string2))
print(len(string3))

In [ ]:
!cp submission.csv /content/drive/MyDrive/datas/submission.csv